In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import joblib
import tensorflow as tf

from scipy.ndimage.interpolation import shift
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D # convolution layers
from keras.layers import Dense, Dropout, Flatten # core layers

from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [2]:
# Read in the data. Label is the numeric label (0-9). Other columns in X data represent the pixel intensity (0-255) of the image
# at the designated pixel. The MNIST data is already quite clean and well pre-processed, so we can just feed the data into
# our model.

train_data = pd.read_csv("mnist_train.csv")
X_train = train_data.drop("label", axis = 1)
y_train = train_data["label"]

test_data = pd.read_csv("mnist_test.csv")
X_test = test_data.drop("label", axis = 1)
y_test = test_data["label"]

X_train = X_train.values.reshape(60000,28,28,1)
X_test = X_test.values.reshape(10000,28,28,1)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [3]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
    
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 117s 62ms/step - loss: 0.1721 - accuracy: 0.9482 - val_loss: 0.0482 - val_accuracy: 0.9848
Epoch 2/10
1875/1875 [==============================] - 116s 62ms/step - loss: 0.0504 - accuracy: 0.9857 - val_loss: 0.0353 - val_accuracy: 0.9898
Epoch 3/10
1875/1875 [==============================] - 115s 61ms/step - loss: 0.0325 - accuracy: 0.9905 - val_loss: 0.0336 - val_accuracy: 0.9892
Epoch 4/10
1875/1875 [==============================] - 120s 64ms/step - loss: 0.0315 - accuracy: 0.9904 - val_loss: 0.0317 - val_accuracy: 0.9904
Epoch 5/10
1875/1875 [==============================] - 121s 65ms/step - loss: 0.0218 - accuracy: 0.9934 - val_loss: 0.0296 - val_accuracy: 0.9926
Epoch 6/10
1875/1875 [==============================] - 116s 62ms/step - loss: 0.0195 - accuracy: 0.9943 - val_loss: 0.0459 - val_accuracy: 0.9880
Epoch 7/10
1875/1875 [==============================] - 117s 62ms/step - loss: 0.0158 - accuracy: 0.9952 - val_loss: 0

In [4]:
y_test_pred = model.predict(X_test)
accuracy = model.evaluate(X_test, y_test)
print (accuracy)

y_test_vals = np.empty(len(y_test_pred), dtype = 'int32')
for ix in range (0, len(y_test_pred)):
    y_test_vals[ix] = np.argmax(y_test_pred[ix])

print (y_test_vals)

313/313 [==============================] - 4s 13ms/step - loss: 0.0228 - accuracy: 0.9945
[0.022849183529615402, 0.9944999814033508]
[7 2 1 ... 4 5 6]


In [5]:

conf_matrix = tf.math.confusion_matrix(
    y_test, y_test_vals, num_classes=10, weights=None, dtype=tf.dtypes.int32,
    name=None
)
print (np.array(conf_matrix))

[[ 978    0    0    0    0    0    1    1    0    0]
 [   0 1132    0    0    0    0    0    3    0    0]
 [   1    0 1026    1    0    0    0    4    0    0]
 [   0    0    0 1007    0    2    0    0    1    0]
 [   0    0    0    0  979    0    0    0    1    2]
 [   1    0    0    4    0  883    1    1    1    1]
 [   6    2    0    0    0    1  949    0    0    0]
 [   0    1    1    0    0    0    0 1023    0    3]
 [   0    0    1    1    0    1    0    0  970    1]
 [   0    0    0    0    7    1    0    1    2  998]]


## CNN Model Evaluation:

Model_v4 utilizes a Convolutional Neural Network. It yields a marginally better accuracy score on the test data (98.8% as opposed to model_v2's 97.1%).

The Confusion Matrix for this model on the test data set (columns from 0-9, representing the classified digit):


    [[ 973    0    3    0    0    0    2    1    1    0]
     [   0 1127    3    0    2    2    0    1    0    0]
     [   1    1 1023    0    1    0    0    5    1    0]
     [   0    0    3 1000    0    4    0    2    1    0]
     [   0    0    0    0  976    0    2    1    0    3]
     [   1    0    0    5    0  882    3    0    1    0]
     [   3    2    0    0    3    5  942    0    3    0]
     [   0    2    6    0    1    0    0 1019    0    0]
     [   2    0    3    1    6    0    2    2  954    4]
     [   1    1    0    2   14    3    0    5    1  982]]

As evidenced by the confusion matrix, model performance on 4's is the worst, often confusing them for 9's. This is understandable since 4's and 9's are visually similar. 

In [6]:
# Save model to file called "KNN_MNIST_ImageClassifier_v4"
model.save("KNN_MNIST_ImageClassifier_v4")


INFO:tensorflow:Assets written to: KNN_MNIST_ImageClassifier_v4\assets
